Python code for merging the three datasets of WalMart-Features, Stores and Train CSVs

In [2]:
import pandas as pd

# Load data
train = pd.read_csv(r"Walmart\train.csv")
features = pd.read_csv(r"Walmart\features.csv")
stores = pd.read_csv(r"Walmart\stores.csv")

# Convert Date to datetime (dayfirst=True because format is dd-mm-yyyy)
train["Date"] = pd.to_datetime(train["Date"], dayfirst=True)
features["Date"] = pd.to_datetime(features["Date"], dayfirst=True)


# Merge train with features (on Store + Date only)
train_features = pd.merge(train, features, on=["Store", "Date"], how="left")

# Merge with stores metadata
master = pd.merge(train_features, stores, on="Store", how="left")

# Sort by Store, Dept, Date
master = master.sort_values(by=["Store", "Dept", "Date"]).reset_index(drop=True)

# Inspect
print(master.head())
print(master.shape)

# Save for reuse
master.to_csv("walmart_master.csv", index=False)



   Store  Dept       Date  Weekly_Sales  IsHoliday_x  Temperature  Fuel_Price  \
0      1     1 2010-02-05      24924.50        False        42.31       2.572   
1      1     1 2010-02-12      46039.49         True        38.51       2.548   
2      1     1 2010-02-19      41595.55        False        39.93       2.514   
3      1     1 2010-02-26      19403.54        False        46.63       2.561   
4      1     1 2010-03-05      21827.90        False        46.50       2.625   

   MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5         CPI  \
0        NaN        NaN        NaN        NaN        NaN  211.096358   
1        NaN        NaN        NaN        NaN        NaN  211.242170   
2        NaN        NaN        NaN        NaN        NaN  211.289143   
3        NaN        NaN        NaN        NaN        NaN  211.319643   
4        NaN        NaN        NaN        NaN        NaN  211.350143   

   Unemployment  IsHoliday_y Type    Size  
0         8.106        False    A  1

Convert Date column

In [2]:
import pandas as pd

# Load the merged dataset
df = pd.read_csv("walmart_master.csv")

# Parse to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Create new time-based features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Week'] = df['Date'].dt.isocalendar().week.astype(int)
df['Day'] = df['Date'].dt.day


Store & Department Filtering


In [5]:
# Check how many unique stores and departments exist
print("Unique Stores:", df['Store'].nunique())
print("Unique Departments:", df['Dept'].nunique())



# Select manageable subset
subset = df[(df['Store'].isin([1, 2])) & (df['Dept'].isin([1, 2, 3]))]

print("Subset Shape:", subset.shape)
print("Stores in subset:", subset['Store'].unique())
print("Departments in subset:", subset['Dept'].unique())


Unique Stores: 45
Unique Departments: 81
Subset Shape: (858, 21)
Stores in subset: [1 2]
Departments in subset: [1 2 3]


Clean Missing Values,

In [16]:
# Check missing values count
print(subset.isnull().sum())
missing_percent = subset.isnull().mean() * 100
print(missing_percent)
# Impute missing values in MarkDown columns with 0
markdown_cols = ["MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"]

for col in markdown_cols:
    df[col] = df[col].fillna(0)

# Verify no missing values remain
print(df.isnull().sum())
for col in markdown_cols:
    df[col + "_missing"] = (df[col] == 0).astype(int)  # since NaNs were filled with 0


df.to_csv("processed_walmart.csv", index=False)


Store             0
Dept              0
Date              0
Weekly_Sales      0
IsHoliday_x       0
Temperature       0
Fuel_Price        0
MarkDown1       552
MarkDown2       606
MarkDown3       558
MarkDown4       552
MarkDown5       552
CPI               0
Unemployment      0
IsHoliday_y       0
Type              0
Size              0
Year              0
Month             0
Week              0
Day               0
dtype: int64
Store            0.000000
Dept             0.000000
Date             0.000000
Weekly_Sales     0.000000
IsHoliday_x      0.000000
Temperature      0.000000
Fuel_Price       0.000000
MarkDown1       64.335664
MarkDown2       70.629371
MarkDown3       65.034965
MarkDown4       64.335664
MarkDown5       64.335664
CPI              0.000000
Unemployment     0.000000
IsHoliday_y      0.000000
Type             0.000000
Size             0.000000
Year             0.000000
Month            0.000000
Week             0.000000
Day              0.000000
dtype: float64
Store 